In [1]:
import sys
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis')

In [2]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import scipy
import importlib
import pickle
import psutil 
import gc

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

load data

In [3]:
training_group = ['eBlinq17', 'eBlinq19a','eBlinq19b','eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay','Pre-Piv-2-training']
# test_group = ['Pre-Piv-2-training']
# training_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqFW231','iBlinqOverlay']
# test_group = ['Pre-Piv-2-training','eBlinqMayStudy']
# test_group = ['CPP-008-27', 'CPP-008-25-Validation']
test_group = ['eBlinqMayStudy']
training_time_window = [0.083,5]
all_training_channel = ['1','3','4']
training_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide','Tier 1w-c: 0/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide','Tier 1w-d: Missing data','Tier 1']
test_tier = ['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide','Tier 1w-c: 0/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide','Tier 1w-d: Missing data','Tier 1']
training_chemistry = '2.6.11'
test_chemistry = '2.6.11'
iso_thresh = 25
feature_hour_short_window = '8hr'
feature_hour_long_window = '120hr'
feature_col = [
    'time_from_start_featureGeneration',
    f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',    
    f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
]
gluc_col = [
    'model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose',
    'biolinq_s1_channelFaultDetection','biolinq_s3_channelFaultDetection','biolinq_s4_channelFaultDetection',
]

gluc_ref_col = [
    'Study','Sensor_Id','Chemistry','Sorting Tier',
    'retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose','cgm_adj','ysi','acck',
    'retro_rmse_s1','retro_rmse_s3','retro_rmse_s4'
]

ground_truth_col = [
    'retro_slope_s1','retro_intercept_s1',
    'retro_slope_s3','retro_intercept_s3',
    'retro_slope_s4','retro_intercept_s4',
]
all_cols = feature_col + gluc_col + gluc_ref_col + ground_truth_col
percentile_lb = 0.01
percentile_ub = 0.99
slope_lb = 0.1
slope_ub = 10
intercept_lb = -100
intercept_ub = 100
rmse_quantile_thresh = 1

In [4]:
df_orig=pd.read_csv(r'A:\Algo\Isabella_v02 model comparison\agg_sample_df_ref_training_validation.csv')

In [5]:
sensor_info_df = pd.read_excel(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Investigate May Study\all study sensor info.xlsx')

In [6]:
df_orig = df_orig.merge(right=sensor_info_df,how='left',left_on='Sensor_Id',right_on='sensor_id',suffixes=(None,'2'))

In [7]:
df_orig['Study'].unique()

array(['eBlinq19c', 'eBlinq20', 'eBlinq22', 'eBlinq23', 'eBlinq25',
       'eBlinqRingOverlay&NiMNA', 'Pre-Piv-2-training', 'iBlinqAcet',
       'iBlinqFW231', 'iBlinqOverlay', 'Pre-Piv-2-validation',
       'eBlinqMayStudy'], dtype=object)

In [8]:
df_orig['Chemistry'].unique()

array(['2.6.11', '2.5.11', nan, 'KCL_IVArmNo', 'KCL Wide_IVArmNo',
       'Control_IVArmYes', 'Pre-spray_IVArmYes', 'Pre-spray_IVArmNo',
       'KCL_IVArmYes', 'KCL Wide_IVArmYes'], dtype=object)

In [9]:
df_orig['Sorting Tier'].unique()

array(['Tier 1: 3/3 Clinical Use', 'Tier 1w-a: 2/3 CLN, 3/3 Wide',
       'Tier 1w-c: 0/3 CLN, 3/3 Wide', 'Tier 1w-b: 1/3 CLN, 3/3 Wide',
       'Tier 1w-d: Missing data', 'Tier 2: 2/3 Pass CLN', nan,
       'Tier 4: 3/3 Connected at Cal', 'Tier 1', 'Trace Review Fail'],
      dtype=object)

Setting

In [10]:
df = df_orig[all_cols].copy()
feature_name_short_avg_ch1 = f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_short_avg_ch1_lb = df[feature_name_short_avg_ch1].quantile(q=percentile_lb)
feature_name_short_avg_ch1_ub = df[feature_name_short_avg_ch1].quantile(q=percentile_ub)
feature_name_short_std_ch1 = f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_short_std_ch1_lb = df[feature_name_short_std_ch1].quantile(q=percentile_lb)
feature_name_short_std_ch1_ub = df[feature_name_short_std_ch1].quantile(q=percentile_ub)
feature_name_short_avg_ch3 = f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_short_avg_ch3_lb = df[feature_name_short_avg_ch3].quantile(q=percentile_lb)
feature_name_short_avg_ch3_ub = df[feature_name_short_avg_ch3].quantile(q=percentile_ub)
feature_name_short_std_ch3 = f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_short_std_ch3_lb = df[feature_name_short_std_ch3].quantile(q=percentile_lb)
feature_name_short_std_ch3_ub = df[feature_name_short_std_ch3].quantile(q=percentile_ub)
feature_name_short_avg_ch4 = f'ewm_average_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_short_avg_ch4_lb = df[feature_name_short_avg_ch4].quantile(q=percentile_lb)
feature_name_short_avg_ch4_ub = df[feature_name_short_avg_ch4].quantile(q=percentile_ub)
feature_name_short_std_ch4 = f'estimate_standard_deviation_{feature_hour_short_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_short_std_ch4_lb = df[feature_name_short_std_ch4].quantile(q=percentile_lb)
feature_name_short_std_ch4_ub = df[feature_name_short_std_ch4].quantile(q=percentile_ub)

feature_name_long_avg_ch1 = f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_long_avg_ch1_lb = df[feature_name_long_avg_ch1].quantile(q=percentile_lb)
feature_name_long_avg_ch1_ub = df[feature_name_long_avg_ch1].quantile(q=percentile_ub)
feature_name_long_std_ch1 = f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_long_std_ch1_lb = df[feature_name_long_std_ch1].quantile(q=percentile_lb)
feature_name_long_std_ch1_ub = df[feature_name_long_std_ch1].quantile(q=percentile_ub)
feature_name_long_avg_ch3 = f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_long_avg_ch3_lb = df[feature_name_long_avg_ch3].quantile(q=percentile_lb)
feature_name_long_avg_ch3_ub = df[feature_name_long_avg_ch3].quantile(q=percentile_ub)
feature_name_long_std_ch3 = f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_long_std_ch3_lb = df[feature_name_long_std_ch3].quantile(q=percentile_lb)
feature_name_long_std_ch3_ub = df[feature_name_long_std_ch3].quantile(q=percentile_ub)
feature_name_long_avg_ch4 = f'ewm_average_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_long_avg_ch4_lb = df[feature_name_long_avg_ch4].quantile(q=percentile_lb)
feature_name_long_avg_ch4_ub = df[feature_name_long_avg_ch4].quantile(q=percentile_ub)
feature_name_long_std_ch4 = f'estimate_standard_deviation_{feature_hour_long_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_long_std_ch4_lb = df[feature_name_long_std_ch4].quantile(q=percentile_lb)
feature_name_long_std_ch4_ub = df[feature_name_long_std_ch4].quantile(q=percentile_ub)

rmse_s1_thresh = df['retro_rmse_s1'].quantile(q=rmse_quantile_thresh)
rmse_s3_thresh = df['retro_rmse_s3'].quantile(q=rmse_quantile_thresh)
rmse_s4_thresh = df['retro_rmse_s4'].quantile(q=rmse_quantile_thresh)

training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                df['Chemistry'].eq(training_chemistry) & df['Sorting Tier'].isin(training_tier)
df_training = df.loc[training_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
valid_training_index = df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                        df['Chemistry'].eq(training_chemistry) & df['Sorting Tier'].isin(training_tier) & \
                        df[feature_name_short_avg_ch1].between(feature_name_short_avg_ch1_lb,feature_name_short_avg_ch1_ub) & \
                        df[feature_name_short_std_ch1].between(feature_name_short_std_ch1_lb,feature_name_short_std_ch1_ub) & \
                        df[feature_name_short_avg_ch3].between(feature_name_short_avg_ch3_lb,feature_name_short_avg_ch3_ub) & \
                        df[feature_name_short_std_ch3].between(feature_name_short_std_ch3_lb,feature_name_short_std_ch3_ub) & \
                        df[feature_name_short_avg_ch4].between(feature_name_short_avg_ch4_lb,feature_name_short_avg_ch4_ub) & \
                        df[feature_name_short_std_ch4].between(feature_name_short_std_ch4_lb,feature_name_short_std_ch4_ub) & \
                        df[feature_name_long_avg_ch1].between(feature_name_long_avg_ch1_lb,feature_name_long_avg_ch1_ub) & \
                        df[feature_name_long_std_ch1].between(feature_name_long_std_ch1_lb,feature_name_long_std_ch1_ub) & \
                        df[feature_name_long_avg_ch3].between(feature_name_long_avg_ch3_lb,feature_name_long_avg_ch3_ub) & \
                        df[feature_name_long_std_ch3].between(feature_name_long_std_ch3_lb,feature_name_long_std_ch3_ub) & \
                        df[feature_name_long_avg_ch4].between(feature_name_long_avg_ch4_lb,feature_name_long_avg_ch4_ub) & \
                        df[feature_name_long_std_ch4].between(feature_name_long_std_ch4_lb,feature_name_long_std_ch4_ub) & \
                        df['retro_slope_s1'].between(slope_lb,slope_ub) & df['retro_intercept_s1'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s3'].between(slope_lb,slope_ub) & df['retro_intercept_s3'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s4'].between(slope_lb,slope_ub) & df['retro_intercept_s4'].between(intercept_lb,intercept_ub) & \
                        df['retro_rmse_s1'].le(rmse_s1_thresh) & df['retro_rmse_s3'].le(rmse_s3_thresh) & df['retro_rmse_s4'].le(rmse_s4_thresh)
df_training_valid = df.loc[valid_training_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
print(f'Training data has shape {df_training.shape}')
print(f'Training data has shape {df_training_valid.shape}')
good_data_ratio = len(df_training_valid)/len(df_training)
print(f'Ratio of training good data is {good_data_ratio}')
x_train = df_training_valid[feature_col]
y_train = df_training_valid[ground_truth_col].to_numpy()
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
x_train_norm = min_max_scaler.fit_transform(x_train)

test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
            df['Chemistry'].eq(test_chemistry) & df['Sorting Tier'].isin(test_tier)
df_test = df.loc[test_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
valid_test_index = df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & \
                        df['Chemistry'].eq(test_chemistry) & df['Sorting Tier'].isin(test_tier) & \
                        df[feature_name_short_avg_ch1].between(feature_name_short_avg_ch1_lb,feature_name_short_avg_ch1_ub) & \
                        df[feature_name_short_std_ch1].between(feature_name_short_std_ch1_lb,feature_name_short_std_ch1_ub) & \
                        df[feature_name_short_avg_ch3].between(feature_name_short_avg_ch3_lb,feature_name_short_avg_ch3_ub) & \
                        df[feature_name_short_std_ch3].between(feature_name_short_std_ch3_lb,feature_name_short_std_ch3_ub) & \
                        df[feature_name_short_avg_ch4].between(feature_name_short_avg_ch4_lb,feature_name_short_avg_ch4_ub) & \
                        df[feature_name_short_std_ch4].between(feature_name_short_std_ch4_lb,feature_name_short_std_ch4_ub) & \
                        df[feature_name_long_avg_ch1].between(feature_name_long_avg_ch1_lb,feature_name_long_avg_ch1_ub) & \
                        df[feature_name_long_std_ch1].between(feature_name_long_std_ch1_lb,feature_name_long_std_ch1_ub) & \
                        df[feature_name_long_avg_ch3].between(feature_name_long_avg_ch3_lb,feature_name_long_avg_ch3_ub) & \
                        df[feature_name_long_std_ch3].between(feature_name_long_std_ch3_lb,feature_name_long_std_ch3_ub) & \
                        df[feature_name_long_avg_ch4].between(feature_name_long_avg_ch4_lb,feature_name_long_avg_ch4_ub) & \
                        df[feature_name_long_std_ch4].between(feature_name_long_std_ch4_lb,feature_name_long_std_ch4_ub) & \
                        df['retro_slope_s1'].between(slope_lb,slope_ub) & df['retro_intercept_s1'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s3'].between(slope_lb,slope_ub) & df['retro_intercept_s3'].between(intercept_lb,intercept_ub) & \
                        df['retro_slope_s4'].between(slope_lb,slope_ub) & df['retro_intercept_s4'].between(intercept_lb,intercept_ub) & \
                        df['retro_rmse_s1'].le(rmse_s1_thresh) & df['retro_rmse_s3'].le(rmse_s3_thresh) & df['retro_rmse_s4'].le(rmse_s4_thresh)
df_test_valid = df.loc[valid_test_index,:].dropna(subset = feature_col + ground_truth_col).reset_index(drop=True)
print(f'Test data has shape {df_test.shape}')
print(f'Test data has shape {df_test_valid.shape}')
good_test_data_ratio = len(df_test_valid)/len(df_test)
print(f'Ratio of good test data is {good_test_data_ratio}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

Training data has shape (5861170, 38)
Training data has shape (5161968, 38)
Ratio of training good data is 0.8807060706309491
Test data has shape (800745, 38)
Test data has shape (750434, 38)
Ratio of good test data is 0.937169760660385


In [11]:
self_cal_model_dict = {}

In [12]:
# For channel 1 slope
i = 0
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=0.0005,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 0
Max of predict_slope is 1.597728465836325
Min of predict_slope is 0.5660254744938662
The model today has MSE=0.06622511022602241
The self cal model has MSE=0.04445012687234812
Max of predict_slope_test is 1.2058618573289084
Min of predict_slope_test is 0.5981223557137284
The model today on test has MSE=0.08989099628624599
The self cal model on test has MSE=0.041603677603988115


In [13]:
# For channel 1 intercept
i = 1
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=0.0005,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 1
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289
The model today has MSE=605.3400784995486
The self cal model has MSE=277.38645520229204
Max of predict_intercept_test is 17.403948291701727
Min of predict_intercept_test is -27.73438743519289
The model today on test has MSE=432.1062222246409
The self cal model on test has MSE=572.844878716862


In [14]:
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_training_valid['model_output_s1_selfcal'] = df_training_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_valid['model_output_s1_selfcal'] = df_test_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

Max of predict_slope is 1.597728465836325
Min of predict_slope is 0.5660254744938662
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289
Training data for ch1 glucose by model today N = 3683418, MRD = -0.049419478946650025, StdRD = 0.29614601807220853, MARD = 0.22118953300334682
Training data for ch1 glucose by selfcal N = 3683418, MRD = -0.021590728776361506, StdRD = 0.29415267797300365, MARD = 0.21807627159253057
Training data for ch1 glucose by retro N = 3683418, MRD = -0.006525732182063572, StdRD = 0.2604195494292999, MARD = 0.16790812429666835
Max of predict_slope is 1.2058618573289084
Min of predict_slope is 0.5981223557137284
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289
Test data for ch1 glucose by model today N = 701247, MRD = -0.22223703408046316, StdRD = 0.23775909770192077, MARD = 0.2671167061201548
Test data for ch1 glucose by selfcal N = 701247, MRD = -0.12440985158373734, StdRD = 0.

In [15]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_train_ysi['model_output_s1_selfcal'] = df_train_ysi['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_today = df_train_ysi['model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_selfcal = df_train_ysi['model_output_s1_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch1_train_retro = df_train_ysi['retro_model_output_s1_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_ysi['model_output_s1_selfcal'] = df_test_ysi['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_today = df_test_ysi['model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_selfcal = df_test_ysi['model_output_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

Max of predict_slope is 1.6181579796064374
Min of predict_slope is 0.5227219269529801
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\2400631413.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch1 glucose by model today N = 184481, MRD = -0.017902213006599957, StdRD = 0.2582203118351232, MARD = 0.1946855099413917
Training data for ch1 glucose by selfcal N = 184481, MRD = 0.027638463530425882, StdRD = 0.2898851276506848, MARD = 0.21788242740951144
Training data for ch1 glucose by retro N = 182978, MRD = 0.03577111376068544, StdRD = 0.23672040181604093, MARD = 0.16979910239717408
Max of predict_slope is 1.2478630202276273
Min of predict_slope is 0.5227287187611644
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289
Test data for ch1 glucose by model today N = 44042, MRD = -0.22569659157780864, StdRD = 0.21951972992760932, MARD = 0.25244884098748405
Test data for ch1 glucose by selfcal N = 44042, MRD = -0.11559411811278365, StdRD = 0.27442071754964625, MARD = 0.23682361714348038
Test data for ch1 glucose by retro N = 44042, MRD = 0.002304656196431923, StdRD = 0.25013300190313786, MARD = 0.17623556003735152


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\2400631413.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_train.max()}')
print(f'Min of predict_slope is {ch1_slope_train.min()}')
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_train.max()}')
print(f'Min of predict_intercept is {ch1_intercept_train.min()}')
df_test_ysi['biolinq_s1_selfcal'] = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(ch1_intercept_train).div(ch1_slope_train)
n_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_train_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch1 glucose by model today N = {n_ch1_train_today}, MRD = {mrd_ch1_train_today}, StdRD = {stdrd_ch1_train_today}, MARD = {mard_ch1_train_today}')
print(f'Training data for ch1 glucose by selfcal N = {n_ch1_train_selfcal}, MRD = {mrd_ch1_train_selfcal}, StdRD = {stdrd_ch1_train_selfcal}, MARD = {mard_ch1_train_selfcal}')
print(f'Training data for ch1 glucose by retro N = {n_ch1_train_retro}, MRD = {mrd_ch1_train_retro}, StdRD = {stdrd_ch1_train_retro}, MARD = {mard_ch1_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch1_slope_test.max()}')
print(f'Min of predict_slope is {ch1_slope_test.min()}')
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch1_intercept_test.max()}')
print(f'Min of predict_intercept is {ch1_intercept_test.min()}')
df_test_ysi['biolinq_s1_selfcal'] = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(ch1_intercept_test).div(ch1_slope_test)
n_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_today = df_test_ysi['biolinq_s1_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_selfcal = df_test_ysi['biolinq_s1_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch1_test_retro = df_test_ysi['retro_model_output_s1_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch1 glucose by model today N = {n_ch1_test_today}, MRD = {mrd_ch1_test_today}, StdRD = {stdrd_ch1_test_today}, MARD = {mard_ch1_test_today}')
print(f'Test data for ch1 glucose by selfcal N = {n_ch1_test_selfcal}, MRD = {mrd_ch1_test_selfcal}, StdRD = {stdrd_ch1_test_selfcal}, MARD = {mard_ch1_test_selfcal}')
print(f'Test data for ch1 glucose by retro N = {n_ch1_test_retro}, MRD = {mrd_ch1_test_retro}, StdRD = {stdrd_ch1_test_retro}, MARD = {mard_ch1_test_retro}')

Max of predict_slope is 1.6181579796064374
Min of predict_slope is 0.5227219269529801
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\1501457173.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch1 glucose by model today N = 169011, MRD = 0.006511552493683683, StdRD = 0.22284512521064104, MARD = 0.1717230698602842
Training data for ch1 glucose by selfcal N = 169011, MRD = 0.048925736072794025, StdRD = 0.26098275735144727, MARD = 0.19954769109832243
Training data for ch1 glucose by retro N = 182978, MRD = 0.03577111376068544, StdRD = 0.23672040181604093, MARD = 0.16979910239717408
Max of predict_slope is 1.2478630202276273
Min of predict_slope is 0.5227287187611644
Max of predict_intercept is 17.403948291701727
Min of predict_intercept is -27.73438743519289
Test data for ch1 glucose by model today N = 39614, MRD = -0.19201181631131736, StdRD = 0.18187142372651005, MARD = 0.2216389180078642
Test data for ch1 glucose by selfcal N = 39614, MRD = -0.08115109561487617, StdRD = 0.25099648217732445, MARD = 0.21406564619932458
Test data for ch1 glucose by retro N = 44042, MRD = 0.002304656196431923, StdRD = 0.25013300190313786, MARD = 0.17623556003735152


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\1501457173.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
[ch1_slope_train.min(),
ch1_slope_train.max(),
ch1_intercept_train.min(),
ch1_intercept_train.max(),
ch1_slope_test.min(),
ch1_slope_test.max(),
ch1_intercept_test.min(),
ch1_intercept_test.max()]

[0.5227219269529801,
 1.6181579796064374,
 -27.73438743519289,
 17.403948291701727,
 0.5227287187611644,
 1.2478630202276273,
 -27.73438743519289,
 17.403948291701727]

In [18]:
# For channel 3 slope
i = 2
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 2
Max of predict_slope is 1.5874109013599917
Min of predict_slope is 0.21944259672586275
The model today has MSE=0.07475921366530094
The self cal model has MSE=0.0467777024379984
Max of predict_slope_test is 1.363025066458246
Min of predict_slope_test is 0.4429913763244384
The model today on test has MSE=0.07230713212830482
The self cal model on test has MSE=0.04935258788599621


In [19]:
# For channel 3 intercept
i = 3
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 3
Max of predict_intercept is 65.25786225417434
Min of predict_intercept is -91.51567842163823
The model today has MSE=717.9167744181003
The self cal model has MSE=233.77150557290358
Max of predict_intercept_test is 60.34792271033385
Min of predict_intercept_test is -94.24502053175237
The model today on test has MSE=607.4122690399154
The self cal model on test has MSE=1014.3744168991809


In [20]:
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_training_valid['model_output_s3_selfcal'] = df_training_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_valid['model_output_s3_selfcal'] = df_test_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

Max of predict_slope is 1.5874109013599917
Min of predict_slope is 0.21944259672586275
Max of predict_intercept is 65.25786225417434
Min of predict_intercept is -91.51567842163823
Training data for ch3 glucose by model today N = 3683418, MRD = -0.10200465017793249, StdRD = 0.2733312748011913, MARD = 0.2219338612948164
Training data for ch3 glucose by selfcal N = 3683418, MRD = -0.02012807269582244, StdRD = 0.3001640908881063, MARD = 0.22290446125776933
Training data for ch3 glucose by retro N = 3683418, MRD = -0.010447200385006057, StdRD = 0.263127317200503, MARD = 0.1666778565142784
Max of predict_slope is 1.363025066458246
Min of predict_slope is 0.4429913763244384
Max of predict_intercept is 60.34792271033385
Min of predict_intercept is -94.24502053175237
Test data for ch3 glucose by model today N = 701247, MRD = -0.21116515786096024, StdRD = 0.24932608239278578, MARD = 0.2685351706037254
Test data for ch3 glucose by selfcal N = 701247, MRD = -0.12464822658811471, StdRD = 0.29472195

In [21]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_train_ysi['model_output_s3_selfcal'] = df_train_ysi['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_today = df_train_ysi['model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_selfcal = df_train_ysi['model_output_s3_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch3_train_retro = df_train_ysi['retro_model_output_s3_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_ysi['model_output_s3_selfcal'] = df_test_ysi['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_today = df_test_ysi['model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_selfcal = df_test_ysi['model_output_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

Max of predict_slope is 1.6267685359457844
Min of predict_slope is -0.3747413285293933
Max of predict_intercept is 65.35547185791862
Min of predict_intercept is -98.32611815856805


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\1433483117.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch3 glucose by model today N = 184481, MRD = -0.06061756139038752, StdRD = 0.243333444744908, MARD = 0.18891265778366836
Training data for ch3 glucose by selfcal N = 184481, MRD = 0.033869438205770805, StdRD = 2.5430273864071564, MARD = 0.24289013619807379
Training data for ch3 glucose by retro N = 182978, MRD = 0.06433215033455573, StdRD = 0.2569939100484107, MARD = 0.1805645447857615
Max of predict_slope is 1.5440547167378926
Min of predict_slope is 0.4429913763244384
Max of predict_intercept is 60.34792271033385
Min of predict_intercept is -94.88913484658121
Test data for ch3 glucose by model today N = 44042, MRD = -0.19906504226935867, StdRD = 0.23780076772625733, MARD = 0.24578955182041193
Test data for ch3 glucose by selfcal N = 44042, MRD = -0.08586128640956067, StdRD = 0.30717056045159347, MARD = 0.24334041703498896
Test data for ch3 glucose by retro N = 44042, MRD = 0.016434520366777505, StdRD = 0.24627390283988235, MARD = 0.1662716834610682


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\1433483117.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_train.max()}')
print(f'Min of predict_slope is {ch3_slope_train.min()}')
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_train.max()}')
print(f'Min of predict_intercept is {ch3_intercept_train.min()}')
df_test_ysi['biolinq_s3_selfcal'] = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(ch3_intercept_train).div(ch3_slope_train)
n_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_train_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch3 glucose by model today N = {n_ch3_train_today}, MRD = {mrd_ch3_train_today}, StdRD = {stdrd_ch3_train_today}, MARD = {mard_ch3_train_today}')
print(f'Training data for ch3 glucose by selfcal N = {n_ch3_train_selfcal}, MRD = {mrd_ch3_train_selfcal}, StdRD = {stdrd_ch3_train_selfcal}, MARD = {mard_ch3_train_selfcal}')
print(f'Training data for ch3 glucose by retro N = {n_ch3_train_retro}, MRD = {mrd_ch3_train_retro}, StdRD = {stdrd_ch3_train_retro}, MARD = {mard_ch3_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch3_slope_test.max()}')
print(f'Min of predict_slope is {ch3_slope_test.min()}')
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch3_intercept_test.max()}')
print(f'Min of predict_intercept is {ch3_intercept_test.min()}')
df_test_ysi['biolinq_s3_selfcal'] = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(ch3_intercept_test).div(ch3_slope_test)
n_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_today = df_test_ysi['biolinq_s3_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_selfcal = df_test_ysi['biolinq_s3_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch3_test_retro = df_test_ysi['retro_model_output_s3_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch3 glucose by model today N = {n_ch3_test_today}, MRD = {mrd_ch3_test_today}, StdRD = {stdrd_ch3_test_today}, MARD = {mard_ch3_test_today}')
print(f'Test data for ch3 glucose by selfcal N = {n_ch3_test_selfcal}, MRD = {mrd_ch3_test_selfcal}, StdRD = {stdrd_ch3_test_selfcal}, MARD = {mard_ch3_test_selfcal}')
print(f'Test data for ch3 glucose by retro N = {n_ch3_test_retro}, MRD = {mrd_ch3_test_retro}, StdRD = {stdrd_ch3_test_retro}, MARD = {mard_ch3_test_retro}')

Max of predict_slope is 1.6267685359457844
Min of predict_slope is -0.3747413285293933
Max of predict_intercept is 65.35547185791862
Min of predict_intercept is -98.32611815856805


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\3716981306.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch3 glucose by model today N = 170165, MRD = -0.044991986603038624, StdRD = 0.2169094762155451, MARD = 0.17175209004653236
Training data for ch3 glucose by selfcal N = 170165, MRD = 0.049998241640354546, StdRD = 2.585891327662774, MARD = 0.22703017841896164
Training data for ch3 glucose by retro N = 182978, MRD = 0.06433215033455573, StdRD = 0.2569939100484107, MARD = 0.1805645447857615
Max of predict_slope is 1.5440547167378926
Min of predict_slope is 0.4429913763244384
Max of predict_intercept is 60.34792271033385
Min of predict_intercept is -94.88913484658121
Test data for ch3 glucose by model today N = 39212, MRD = -0.16502821194444553, StdRD = 0.1989722072246235, MARD = 0.21350310744063591
Test data for ch3 glucose by selfcal N = 39212, MRD = -0.05341978494591062, StdRD = 0.28748697087205194, MARD = 0.22211546208728977
Test data for ch3 glucose by retro N = 44042, MRD = 0.016434520366777505, StdRD = 0.24627390283988235, MARD = 0.1662716834610682


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\3716981306.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
[ch3_slope_train.min(),
ch3_slope_train.max(),
ch3_intercept_train.min(),
ch3_intercept_train.max(),
ch3_slope_test.min(),
ch3_slope_test.max(),
ch3_intercept_test.min(),
ch3_intercept_test.max()]

[-0.3747413285293933,
 1.6267685359457844,
 -98.32611815856805,
 65.35547185791862,
 0.4429913763244384,
 1.5440547167378926,
 -94.88913484658121,
 60.34792271033385]

In [24]:
# For channel 4 slope
i = 4
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope = self_cal_model.predict(x_train_norm)
print(f'Max of predict_slope is {predict_slope.max()}')
print(f'Min of predict_slope is {predict_slope.min()}')
# predict_slope[predict_slope<0.12] = 1
# predict_slope[predict_slope>2.4] = 1
mse_train_selfcal = np.mean((predict_slope - y_train[:,i])**2)
mse_train_today = np.mean((1-y_train[:,i])**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_slope_test is {predict_slope_test.max()}')
print(f'Min of predict_slope_test is {predict_slope_test.min()}')
# predict_slope_test[predict_slope_test<0.12] = 1
# predict_slope_test[predict_slope_test>2.4] = 1    
mse_test_selfcal = np.mean((predict_slope_test - y_test[:,i])**2)
mse_test_today = np.mean((1-y_test[:,i])**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 4
Max of predict_slope is 1.8983011168759636
Min of predict_slope is 0.5086272669597429
The model today has MSE=0.07027385889051589
The self cal model has MSE=0.044172600365351804
Max of predict_slope_test is 1.344316720740134
Min of predict_slope_test is 0.450869801356102
The model today on test has MSE=0.07583450104308291
The self cal model on test has MSE=0.04844089701946498


In [25]:
# For channel 4 intercept
i = 5
print(f'Optimizing parameter {i}')
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10,1),activation='logistic',solver='adam',alpha=5e-4,max_iter=int(1e4),tol=1e-4,learning_rate='adaptive',random_state=20140806)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_intercept = self_cal_model.predict(x_train_norm)
print(f'Max of predict_intercept is {predict_intercept.max()}')
print(f'Min of predict_intercept is {predict_intercept.min()}')    
# predict_intercept[predict_intercept<-100] = 0
# predict_intercept[predict_intercept>100] = 0   
mse_train_selfcal = np.mean((predict_intercept - y_train[:,i])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_intercept_test = self_cal_model.predict(x_test_norm)
print(f'Max of predict_intercept_test is {predict_intercept_test.max()}')
print(f'Min of predict_intercept_test is {predict_intercept_test.min()}')    
# predict_intercept_test[predict_intercept_test<-100] = 0
# predict_intercept_test[predict_intercept_test>100] = 0     
mse_test_selfcal = np.mean((predict_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 5
Max of predict_intercept is 16.834344364377174
Min of predict_intercept is -33.43589769809133
The model today has MSE=642.8480669047087
The self cal model has MSE=312.860982501751
Max of predict_intercept_test is 16.834344364378026
Min of predict_intercept_test is -33.43589593317621
The model today on test has MSE=602.6277479771732
The self cal model on test has MSE=814.6611812279725


In [26]:
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_training_valid['model_output_s4_selfcal'] = df_training_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
n_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).count()
mrd_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).mean()
stdrd_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).std()
mard_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_valid['model_output_s4_selfcal'] = df_test_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
n_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).count()
mrd_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).mean()
stdrd_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).std()
mard_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

Max of predict_slope is 1.8983011168759636
Min of predict_slope is 0.5086272669597429
Max of predict_intercept is 16.834344364377174
Min of predict_intercept is -33.43589769809133
Training data for ch4 glucose by model today N = 3683418, MRD = -0.033679562986087215, StdRD = 0.29566973010287495, MARD = 0.21815429839098344
Training data for ch4 glucose by selfcal N = 3683418, MRD = -0.027012302026538818, StdRD = 0.2827098746848216, MARD = 0.2095471343568962
Training data for ch4 glucose by retro N = 3683418, MRD = -0.007260714811320242, StdRD = 0.23788755735666078, MARD = 0.15668256895021226
Max of predict_slope is 1.344316720740134
Min of predict_slope is 0.450869801356102
Max of predict_intercept is 16.834344364378026
Min of predict_intercept is -33.43589593317621
Test data for ch4 glucose by model today N = 701247, MRD = -0.1829878192118541, StdRD = 0.26413065526251656, MARD = 0.25599612308042957
Test data for ch4 glucose by selfcal N = 701247, MRD = -0.12479172231742111, StdRD = 0.29

In [27]:
df = df_orig[all_cols].copy()

df_train_ysi = df.loc[training_index]
x_train_ysi = df_train_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_train_ysi['model_output_s4_selfcal'] = df_train_ysi['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_today = df_train_ysi['model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_selfcal = df_train_ysi['model_output_s4_selfcal'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
n_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).count()
mrd_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).mean()
stdrd_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).std()
mard_ch4_train_retro = df_train_ysi['retro_model_output_s4_calculateGlucose'].sub(df_train_ysi['ysi']).div(df_train_ysi['ysi']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_ysi['model_output_s4_selfcal'] = df_test_ysi['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_today = df_test_ysi['model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_selfcal = df_test_ysi['model_output_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

Max of predict_slope is 2.7750485207071933
Min of predict_slope is 0.2239018197824201
Max of predict_intercept is 16.834344364378047
Min of predict_intercept is -33.43589769809133


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\2843018395.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch4 glucose by model today N = 184481, MRD = -0.005735204930328106, StdRD = 0.24809057191950848, MARD = 0.182932500706589
Training data for ch4 glucose by selfcal N = 184481, MRD = 0.020104265440134755, StdRD = 0.2804691041503615, MARD = 0.20178063211234798
Training data for ch4 glucose by retro N = 182978, MRD = 0.052945672894828204, StdRD = 0.23238008015546718, MARD = 0.16899180407373
Max of predict_slope is 2.652774406942605
Min of predict_slope is 0.38946015036209153
Max of predict_intercept is 16.834344364378026
Min of predict_intercept is -33.43589768495777
Test data for ch4 glucose by model today N = 44042, MRD = -0.14730667663510766, StdRD = 0.2711408315224174, MARD = 0.23330688104352823
Test data for ch4 glucose by selfcal N = 44042, MRD = -0.10758289457068813, StdRD = 0.29955184771235704, MARD = 0.23419112035521458
Test data for ch4 glucose by retro N = 44042, MRD = 0.025645517108647282, StdRD = 0.2670910489119329, MARD = 0.17933103194737057


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\2843018395.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
df = df_orig[all_cols].copy()

df_test_ysi = df.loc[training_index]
x_train_ysi = df_test_ysi[feature_col]
x_train_ysi_norm = min_max_scaler.transform(x_train_ysi)

ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_train.max()}')
print(f'Min of predict_slope is {ch4_slope_train.min()}')
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_train.max()}')
print(f'Min of predict_intercept is {ch4_intercept_train.min()}')
df_test_ysi['biolinq_s4_selfcal'] = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(ch4_intercept_train).div(ch4_slope_train)
n_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_train_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Training data for ch4 glucose by model today N = {n_ch4_train_today}, MRD = {mrd_ch4_train_today}, StdRD = {stdrd_ch4_train_today}, MARD = {mard_ch4_train_today}')
print(f'Training data for ch4 glucose by selfcal N = {n_ch4_train_selfcal}, MRD = {mrd_ch4_train_selfcal}, StdRD = {stdrd_ch4_train_selfcal}, MARD = {mard_ch4_train_selfcal}')
print(f'Training data for ch4 glucose by retro N = {n_ch4_train_retro}, MRD = {mrd_ch4_train_retro}, StdRD = {stdrd_ch4_train_retro}, MARD = {mard_ch4_train_retro}')

df_test_ysi = df.loc[test_index]
x_test_ysi = df_test_ysi[feature_col]
x_test_ysi_norm = min_max_scaler.transform(x_test_ysi)
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_ysi_norm)
print(f'Max of predict_slope is {ch4_slope_test.max()}')
print(f'Min of predict_slope is {ch4_slope_test.min()}')
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_ysi_norm)
print(f'Max of predict_intercept is {ch4_intercept_test.max()}')
print(f'Min of predict_intercept is {ch4_intercept_test.min()}')
df_test_ysi['biolinq_s4_selfcal'] = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(ch4_intercept_test).div(ch4_slope_test)
n_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_today = df_test_ysi['biolinq_s4_channelFaultDetection'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_selfcal = df_test_ysi['biolinq_s4_selfcal'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
n_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).count()
mrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).mean()
stdrd_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).std()
mard_ch4_test_retro = df_test_ysi['retro_model_output_s4_calculateGlucose'].sub(df_test_ysi['ysi']).div(df_test_ysi['ysi']).abs().mean()
print(f'Test data for ch4 glucose by model today N = {n_ch4_test_today}, MRD = {mrd_ch4_test_today}, StdRD = {stdrd_ch4_test_today}, MARD = {mard_ch4_test_today}')
print(f'Test data for ch4 glucose by selfcal N = {n_ch4_test_selfcal}, MRD = {mrd_ch4_test_selfcal}, StdRD = {stdrd_ch4_test_selfcal}, MARD = {mard_ch4_test_selfcal}')
print(f'Test data for ch4 glucose by retro N = {n_ch4_test_retro}, MRD = {mrd_ch4_test_retro}, StdRD = {stdrd_ch4_test_retro}, MARD = {mard_ch4_test_retro}')

Max of predict_slope is 2.7750485207071933
Min of predict_slope is 0.2239018197824201
Max of predict_intercept is 16.834344364378047
Min of predict_intercept is -33.43589769809133


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\789112783.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Training data for ch4 glucose by model today N = 173504, MRD = 0.010087768483269985, StdRD = 0.21918615896264948, MARD = 0.16645986629456955
Training data for ch4 glucose by selfcal N = 173504, MRD = 0.03785564703131243, StdRD = 0.2583853009965412, MARD = 0.19017721165226195
Training data for ch4 glucose by retro N = 182978, MRD = 0.052945672894828204, StdRD = 0.23238008015546718, MARD = 0.16899180407373
Max of predict_slope is 2.652774406942605
Min of predict_slope is 0.38946015036209153
Max of predict_intercept is 16.834344364378026
Min of predict_intercept is -33.43589768495777
Test data for ch4 glucose by model today N = 39728, MRD = -0.10719177651440225, StdRD = 0.22744060293808482, MARD = 0.2002691122812465
Test data for ch4 glucose by selfcal N = 39728, MRD = -0.06771614928166209, StdRD = 0.25117529545992034, MARD = 0.20265504029699616
Test data for ch4 glucose by retro N = 44042, MRD = 0.025645517108647282, StdRD = 0.2670910489119329, MARD = 0.17933103194737057


C:\Users\lwang\AppData\Local\Temp\6\ipykernel_33904\789112783.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
[ch4_slope_train.min(),
ch4_slope_train.max(),
ch4_intercept_train.min(),
ch4_intercept_train.max(),
ch4_slope_test.min(),
ch4_slope_test.max(),
ch4_intercept_test.min(),
ch4_intercept_test.max()]

[0.2239018197824201,
 2.7750485207071933,
 -33.43589769809133,
 16.834344364378047,
 0.38946015036209153,
 2.652774406942605,
 -33.43589768495777,
 16.834344364378026]

In [ ]:
df_test_valid[['model_output_s1_selfcal','model_output_s3_selfcal','model_output_s4_selfcal']].std(1).quantile(q=0.95)

In [ ]:
df_test_valid[['model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

In [ ]:
df_test_valid[['retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

In [ ]:
min_max_scaler.data_min_

In [ ]:
min_max_scaler.feature_names_in_ = np.array(['time_from_start',
                    'ewm_average_8hr_iir_filt_model_output_temp_s1',
                    'ewm_average_8hr_iir_filt_model_output_temp_s3',
                    'ewm_average_8hr_iir_filt_model_output_temp_s4',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s1',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s3',
                    'estimate_standard_deviation_8hr_iir_filt_model_output_temp_s4'
                    ],dtype='object')
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\min_max_scaler.p','wb') as fp:
    pickle.dump(min_max_scaler,fp) 

In [ ]:
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch1_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_0'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch1_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_1'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch3_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_2'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch3_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_3'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch4_slope.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_4'],fp)  
with open (r'C:\Users\lwang\Documents\GitHub\DataAnalysis\AlgorithmAssembler\Modules\calculateGlucose\selfcal_nn_models\ch4_intercept.p','wb') as fp:
    pickle.dump(self_cal_model_dict['self_cal_model_5'],fp)                  

In [ ]:
self_cal_model_temp = MLPRegressor(solver='sgd',alpha=1e-1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528,verbose=True)
param_list = {"hidden_layer_sizes": [(10,),(10,10),(10,5,2),(2,5,10),(4,3,4),(3,2,3),(5,5),(5,)]}
gridCV = GridSearchCV(estimator=self_cal_model_temp, param_grid=param_list,cv=10,verbose=True)
gridCV.fit(x_train_norm,y_train[:,i])

In [ ]:
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/min_max_scaler.p','rb') as fp:
#     min_max_scaler = pickle.load(fp)
# min_max_scaler.feature_names_in_ = np.array(['time_from_start_featureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s1_glucoseFeatureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',
#        'ewm_average_8hr_iir_filt_model_output_s4_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s1_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',
#        'estimate_standard_deviation_8hr_iir_filt_model_output_s4_glucoseFeatureGeneration'],
#       dtype='object')

In [ ]:
# self_cal_model_dict = {}
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_0'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch1_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_1'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_2'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch3_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_3'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_slope.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_4'] = pickle.load(fp)
# with open (r'/Users/liangwang/GitHub/DataAnalysis/AlgorithmAssembler/Modules/calculateGlucose/selfcal_nn_models/ch4_intercept.p','rb') as fp:
#     self_cal_model_dict['self_cal_model_5'] = pickle.load(fp)                

In [ ]:
# for i in range(6): # [0,1]:
#     print(f'Loading parameter {i}')
#     self_cal_model = self_cal_model_dict[f'self_cal_model_{i}']
#     predict_slope_intercept = self_cal_model.predict(x_train_norm)
#     mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,i])**2)
#     mse_train_today = np.mean(y_train[:,i]**2)
#     print(f'The model today has MSE={mse_train_today}')
#     print(f'The self cal model has MSE={mse_train_selfcal}')

#     predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
#     mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
#     mse_test_today = np.mean(y_test[:,i]**2)
#     print(f'The model today on test has MSE={mse_test_today}')
#     print(f'The self cal model on test has MSE={mse_test_selfcal}')
#     self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
pd.DataFrame(gridCV.cv_results_)

In [ ]:
gridCV.best_estimator_

In [ ]:
i=1
self_cal_model = MLPRegressor(hidden_layer_sizes=(1,),solver='sgd',alpha=0.1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope_intercept = self_cal_model.predict(x_train_norm)
mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,0])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
y_train[:,i].mean()

In [ ]:
df_training_valid[feature_col].plot(y='ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',kind='hist',bins=1000)

In [ ]:
min_max_scaler.data_max_

In [ ]:
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10),solver='sgd',alpha=1e-4,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,0])
predict_slope_intercept = self_cal_model.predict(x_train_norm)

In [ ]:
valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
                        df[f'iso_s{ch_str}'].le(iso_thresh) & df[f'iso_s3'].le(iso_thresh) & df[f'iso_s4'].le(iso_thresh)].index
df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
print(f'Test data has shape {df_test_valid.shape}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.fit_transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)